<a href="https://colab.research.google.com/github/Aastha031295/FineTuning/blob/main/FineTune_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install the necessary libraries
!pip install transformers datasets bitsandbytes peft

# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

# Load dataset
import pandas as pd

# Provide the correct path to your file (check that this path is accurate after mounting)
dataset_path = '/content/drive/MyDrive/combined_dataset.csv'  # Adjust 'MyDrive' if necessary
df = pd.read_csv(dataset_path)

# Check the data types of the columns
print(df.dtypes)

# Check for missing or unusual values in the label column
print(df['label'].unique())  # Replace 'label' with the actual label column name

# Prepare the dataset for Hugging Face's transformers
from datasets import Dataset

# Convert the dataframe to a Hugging Face Dataset (adjust column names as needed)
train_data = Dataset.from_pandas(df[['text', 'label']])
train_data = train_data.train_test_split(test_size=0.1)

# Load GPT-2 tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# GPT-2 doesn't have a padding token by default, so we add it
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)

# Preprocess the data by tokenizing inputs
def preprocess_function(examples):
    inputs = examples['text']
    model_inputs = tokenizer(inputs, max_length=512, padding='max_length', truncation=True)

    # Labels are the same as the inputs for causal LM
    model_inputs['labels'] = model_inputs['input_ids'].copy()
    return model_inputs

# Tokenize the dataset
tokenized_dataset = train_data.map(preprocess_function, batched=True, remove_columns=['text', 'label'])

# Define the training arguments
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_steps=500,
    save_total_limit=2,
    report_to="none",  # Disable logging to external services like WandB or Huggingface Hub
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy = "epoch" # Add this line
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ib

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.790500,1.752018


Epoch,Training Loss,Validation Loss
1,1.790500,1.752018
2,1.733000,1.740775
3,1.751000,1.737953


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=8814, training_loss=1.7652923907718612, metrics={'train_runtime': 11168.6831, 'train_samples_per_second': 6.312, 'train_steps_per_second': 0.789, 'total_flos': 1.8420304379904e+16, 'train_loss': 1.7652923907718612, 'epoch': 3.0})

In [2]:
# Evaluate the fine-tuned model
results = trainer.evaluate()

# Print the evaluation results
print("Evaluation Results:", results)

Evaluation Results: {'eval_loss': 1.7379528284072876, 'eval_runtime': 119.6555, 'eval_samples_per_second': 21.821, 'eval_steps_per_second': 2.733, 'epoch': 3.0}


In [3]:
model.save_pretrained("/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm")
tokenizer.save_pretrained("/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm")

('/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/tokenizer_config.json',
 '/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/special_tokens_map.json',
 '/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/vocab.json',
 '/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/merges.txt',
 '/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/added_tokens.json',
 '/content/drive/MyDrive/GPT2/finetuned_GPT2_sarcasm/tokenizer.json')